## Kommunikáljunk dokumentumainkkal a `LangChain` & `OpenAI` segítségével

- Fájl: WARRANT TO PURCHASE SHARES OF CAPITAL STOCK
- Feldolgozó algoritmus: GPT-4 (`gpt-4-0314`)

In [14]:
import credentials
import time
import re
import os
os.environ["OPENAI_API_KEY"] = credentials.openai_api

import tiktoken

from langchain.document_loaders import PyPDFLoader

from langchain.chains.question_answering import load_qa_chain
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

### 1. QA (kérdezzünk a szövegről!)

In [2]:
loader = PyPDFLoader("../docs/Law_Insider_eidos-therapeutics-inc_contract_Filed_23-03-2018_Contract.pdf")
docs = loader.load_and_split()

In [11]:
chain = load_qa_chain(ChatOpenAI(model_name = 'gpt-4', temperature=0.0), 
                      chain_type="stuff") #map_reduce

Milyen árazásra számítsunk?

GPT-4
- prompt:       $0.03 / 1K tokens
- completion:   $0.06 / 1K tokens

In [37]:
gpt_4_token_encoding = tiktoken.get_encoding("cl100k_base")
teljes_szoveg = ' '.join([i.page_content for i in docs])
teljes_szoveg_tokenek = gpt_4_token_encoding.encode(teljes_szoveg)

prompt_template = """Use the following pieces of context to answer the users question. 
                     If you don't know the answer, just say that you don't know, don't try to make up an answer"""
prompt_tokenek = gpt_4_token_encoding.encode(prompt_template)

print(len(teljes_szoveg_tokenek))
print(len(prompt_tokenek))

4533
38


- Input tokenek száma: 4533 + 38 = 4571
- Output (completion / válasz) tokenek becslése: ~100

- Teljes költség = (4571 * 0.03 / 1000) + (100 * 0.06 / 1000) = 0.14313$ = 14.3 cent / megválaszolt kérdés
- Ebből 
    - prompt: 13.7 cent
    - completion: 0.6 cent

In [12]:
queries = ['What is this input file?',
           'What is this document about?',
           'What is the title of the contract?',
           'Who are the agreeing parties?',
           'Which law governs the contract?',
           'How many shares and at what price can be purchased?',
           'Upon net exercising, what is the formula used?',
           'Who are the representatives who signed the contract?',
           'Are there any attachments, amendments or any supplementary material included?',
           'When was Michael Jordan born?']

In [13]:
for query in queries:
    print('Q:', query)
    print('\n')
    print(chain.run(input_documents = docs, question = query))
    print('\n', '-------' * 10)
    time.sleep(20)

Question: What is this input file?
Answer:
This input file is a legal document representing a Warrant to Purchase Shares of Capital Stock issued by Eidos Therapeutics, Inc. to BridgeBio Pharma LLC. The document outlines the terms and conditions for the warrant, including the number of shares, exercise price, exercise period, method of exercise, adjustments, and other related provisions. It also includes a Notice of Exercise and Assignment Form for the holder to exercise or assign the warrant.

 ----------------------------------------------------------------------
Question: What is this document about?
Answer:
This document is a Warrant to Purchase Shares of Capital Stock issued by Eidos Therapeutics, Inc. to BridgeBio Pharma LLC. It outlines the terms and conditions for the holder (BridgeBio Pharma LLC) to purchase a specified number of shares of the company's capital stock at a predetermined exercise price. The document also includes provisions for adjustments to the exercise price a

Minden válasz helyesnek bizonyul, ráadásul a "kakukktojás" kérdésen sem hasalt el: kontextusból nem szabadna tudnia, mikor született Michael Jordan, és azt nem is válaszolja meg. Természeseten erre a kérdésre biztosan tudja egyébként a választ, viszont ha nem tudná, 'hallucinálni' kezdene, amit így elkerültünk (prompt-ban utasítottuk, hogy ha kizárólag a kontextusból nem tudja kivenni az infót, akkor mondja meg)

Mielőtt a hivatalos 'szövegösszegző' / 'szövegkivonatoló' promptot használnánk, nézzük meg, hogy a QA is remekül kinyeri a kellő információt.

In [50]:
q = 'Please summarize the entire text as consicely as possible.'
a = chain.run(input_documents = docs, question = q)

In [54]:
print('Q:', q)
print('\n')
#print(re.sub("(.{89})", "\\1\n", a, 0, re.DOTALL))
print(a)

Question: Please summarize the entire text as consicely as possible.
Answer:
This document is a Warrant to Purchase Shares of Capital Stock issued by Eidos Therapeutics, Inc. to BridgeBio Pharma LLC on February 22, 2018. The warrant allows the holder to purchase up to a certain number of fully paid and nonassessable shares of the company's capital stock at a specified exercise price, subject to adjustments and conditions. The exercise period commences upon a Qualified Financing or immediately prior to a Deemed Liquidation Event and ends three years after the commencement of exercisability or upon a Deemed Liquidation Event or IPO. The holder may exercise the warrant in whole or in part, and the company is required to issue the corresponding shares upon exercise. The warrant also includes provisions for adjustments in the event of stock splits, dividends, reclassifications, or other changes in the company's capital stock. The document is governed by Delaware General Corporation Law and 

Egészen jó kivonata az eredetileg 9 oldalas PDF-nek!

### 2. Szövegkivonatolás

In [55]:
chain = load_summarize_chain(ChatOpenAI(model_name = 'gpt-4', temperature=0.0), chain_type="stuff")
summary = chain.run(docs)

In [57]:
print(re.sub("(.{90})", "\\1\n", summary, 0, re.DOTALL))

This document is a warrant issued by Eidos Therapeutics, Inc. to BridgeBio Pharma LLC, gra
nting BridgeBio the right to purchase shares of Eidos' capital stock. The warrant is exerc
isable under certain conditions and subject to adjustments. The warrant is not registered 
under the Securities Act of 1933 and cannot be sold or transferred without proper registra
tion or exemption. The exercise period for the warrant begins on February 22, 2018, and en
ds on February 21, 2021.


Egyedi `prompt` segítségével személyre szabhatjuk az instrukciót!

In [58]:
prompt_template = """Write a concise summary of the following. 
Focus on including financial data, such as number and price of stocks.

{text}

CONCISE SUMMARY:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
chain = load_summarize_chain(ChatOpenAI(model_name = 'gpt-4', temperature=0.0), 
                             chain_type="stuff", prompt=PROMPT)
summary = chain.run(docs)

In [59]:
print(re.sub("(.{90})", "\\1\n", summary, 0, re.DOTALL))

Eidos Therapeutics has issued a warrant to BridgeBio Pharma, allowing the purchase of up t
o $4 million worth of Eidos' capital stock. The warrant is exercisable for shares of New P
referred Stock at the lowest per-share purchase price at which shares of the New Preferred
 Stock are sold in a Qualified Financing at the time of conversion of the Notes. The warra
nt is valid from February 22, 2018, to February 21, 2021.


Nézzük meg mire képes ha magyar outputot kérünk tőle

In [60]:
prompt_template = """Write a concise summary of the following. 
Focus on including financial data, such as number and price of stocks.

{text}

CONCISE SUMMARY IN HUNGARIAN:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
chain = load_summarize_chain(ChatOpenAI(model_name = 'gpt-4', temperature=0.0), 
                             chain_type="stuff", prompt=PROMPT)
summary = chain.run(docs)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=60).


In [61]:
print(re.sub("(.{90})", "\\1\n", summary, 0, re.DOTALL))

Az Eidos Therapeutics, Inc. kibocsát egy részvényvásárlási jogot a BridgeBio Pharma LLC ré
szére, amelynek értéke 4 000 000 dollár. A részvények vételi ára a következőképpen alakul:
 ha az új preferált részvényekre vonatkozik, akkor a minősített finanszírozás során értéke
sített új preferált részvények legalacsonyabb ára; ha a sorozat Seed Preferred részvényekr
e vonatkozik, akkor 1,3248 dollár részvényenként. A részvényvásárlási jog a kibocsátás dát
umától számított három évig érvényes.
